In [1]:
import nest
import numpy as np
import matplotlib.pyplot as plt
nest.ResetKernel()


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.7.0
 Built: May 24 2024 10:11:53

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.



In [12]:
import nest
import matplotlib.pyplot as plt

def create_populations():
    E_neuron_params = {
        "E_L": 0.0,
        "C_m": 1.0,
        "V_T": -63.0,
        "t_ref": 5.0,
        "tau_syn_ex": 5.0,
        "tau_syn_in": 5.0,
        "g_L": 0.3,
        "Act_m" : 0.5,
        "Inact_h" : 0.6,
        "Act_n" : 0.7
    }

    pop_size_E = 80
    pop_size_I = 20

    E_pops = nest.Create("hh_cond_exp_traub", n=pop_size_E, params=E_neuron_params)
    I_pops = nest.Create("hh_cond_exp_traub", n=pop_size_I, params=E_neuron_params)

    return E_pops, I_pops

def connect_pop(E_pops, I_pops):
    """
    Connect excitatory and inhibitory populations with specified synapse properties.
    """
    all_neurons = E_pops + I_pops

    weights_E = 1.0
    weights_I = -0.05
    nest.CopyModel("stdp_synapse", "e_syn", {"Wmax": 2 * weights_E})
    nest.CopyModel("static_synapse", "i_syn")

    connection_type_dict = {"rule": "fixed_total_number", "N": 100}
    syn_e_dict = {"synapse_model": "e_syn", "weight": weights_E, "delay": 1.0}
    syn_i_dict = {"synapse_model": "i_syn", "weight": weights_I, "delay": 1.0}

    nest.Connect(E_pops, all_neurons, connection_type_dict, syn_spec=syn_e_dict)
    nest.Connect(I_pops, all_neurons, connection_type_dict, syn_spec=syn_i_dict)

    return E_pops, I_pops

def prune(threshold=0.5):
    """
    Prunes synapses with weights below a given threshold.
    """
    connections = nest.GetConnections()
    connection_details = nest.GetStatus(connections, keys=["source", "target", "weight"])
    print(connection_details)

    # connections_to_prune = [conn for conn in connection_details if conn[2] < threshold]
    # print(connections_to_prune)
    # for conn in connections_to_prune:
        # nest.Disconnect([conn[0]], [conn[1]])

    # print(f"Pruned {len(connections_to_prune)}")

def create_thalamic_input(E_pops):
    thalamic_input = nest.Create("poisson_generator", params={"rate": 200.0})  
    nest.Connect(thalamic_input, E_pops, syn_spec={"weight": 1.0, "delay": 1.0})
    

def recording_devices(E_pops, I_pops):
    multimeter = nest.Create("multimeter", params={"record_from": ["V_m"]})
    spikerecorder = nest.Create("spike_recorder")
    nest.Connect(multimeter, E_pops + I_pops)
    nest.Connect(E_pops + I_pops, spikerecorder)
    return multimeter, spikerecorder

def plot_spikes(spikerecorder):
    spike_events = nest.GetStatus(spikerecorder, "events")[0]
    if len(spike_events["times"]) > 0:
        plt.figure()
        plt.scatter(spike_events["times"], spike_events["senders"], s=1)
        plt.xlabel("Time (ms)")
        plt.ylabel("Neuron ID")
        plt.title("Spike Raster Plot")
        plt.show()



def plot_results(multimeter, spikerecorder):
    spike_events = nest.GetStatus(spikerecorder, "events")[0]
    voltages = nest.GetStatus(multimeter, "events")[0]

    plt.figure(figsize=(10, 5))
    plt.subplot(2, 1, 1)
    plt.scatter(spike_events["times"], spike_events["senders"], s=1)
    plt.xlabel("Time (ms)")
    plt.ylabel("Neuron ID")
    plt.title("Spike Raster Plot")

    plt.subplot(2, 1, 2)
    for neuron_id in np.unique(voltages["senders"]):
        neuron_voltages = voltages["V_m"][voltages["senders"] == neuron_id]
        neuron_times = voltages["times"][voltages["senders"] == neuron_id]
        plt.plot(neuron_times, neuron_voltages, label=f"Neuron {neuron_id}")
    plt.xlabel("Time (ms)")
    plt.ylabel("Membrane Potential (mV)")
    plt.title("Membrane Potentials")
    plt.legend()
    plt.tight_layout()
    plt.show()

def log_weights(time):
    connections = nest.GetConnections()
    connection_details = nest.GetStatus(connections, keys=["source", "target", "weight"])
    print(f"Weights at time {time} ms:")
    for conn in connection_details:
        print(f"Source: {conn[0]}, Target: {conn[1]}, Weight: {conn[2]}")

def create_and_simulate(simulation_time=1000.0):
    E_pops, I_pops = create_populations()
    connect_pop(E_pops, I_pops)
    create_thalamic_input(E_pops)
    multimeter, spikerecorder = recording_devices(E_pops, I_pops)
    
    log_weights(0)
    
    nest.Simulate(simulation_time)
    
    log_weights(simulation_time)
    
    return multimeter, spikerecorder





In [14]:
nest.ResetKernel()
create_and_simulate(simulation_time=1000.0)

Weights at time 0 ms:
Source: 1, Target: 103, Weight: 1.0
Source: 2, Target: 103, Weight: 1.0
Source: 3, Target: 103, Weight: 1.0
Source: 4, Target: 103, Weight: 1.0
Source: 5, Target: 103, Weight: 1.0
Source: 6, Target: 103, Weight: 1.0
Source: 7, Target: 103, Weight: 1.0
Source: 8, Target: 103, Weight: 1.0
Source: 9, Target: 103, Weight: 1.0
Source: 10, Target: 103, Weight: 1.0
Source: 11, Target: 103, Weight: 1.0
Source: 12, Target: 103, Weight: 1.0
Source: 13, Target: 103, Weight: 1.0
Source: 14, Target: 103, Weight: 1.0
Source: 15, Target: 103, Weight: 1.0
Source: 16, Target: 103, Weight: 1.0
Source: 17, Target: 103, Weight: 1.0
Source: 18, Target: 103, Weight: 1.0
Source: 19, Target: 103, Weight: 1.0
Source: 20, Target: 103, Weight: 1.0
Source: 21, Target: 103, Weight: 1.0
Source: 22, Target: 103, Weight: 1.0
Source: 23, Target: 103, Weight: 1.0
Source: 24, Target: 103, Weight: 1.0
Source: 25, Target: 103, Weight: 1.0
Source: 26, Target: 103, Weight: 1.0
Source: 27, Target: 103, 

(NodeCollection(metadata=None, model=multimeter, size=1, first=102),
 NodeCollection(metadata=None, model=spike_recorder, size=1, first=103))